In [ ]:
import os

# Ruta del directorio donde están los archivos
directorio = r'C:\Users\curso\Downloads\Registro de grabaciones al 28-06-2025\Registro de grabaciones\Tristeza'
nuevo_nombre_base = 'Tristeza'  # nombre base: archivo_1, archivo_2...

# Obtener lista de archivos (sin directorios)
archivos = [f for f in os.listdir(directorio) if os.path.isfile(os.path.join(directorio, f))]
archivos.sort()  # ordena alfabéticamente (opcional)

# Renombrar con contador
for i, nombre_actual in enumerate(archivos, start=1):
    _, extension = os.path.splitext(nombre_actual)
    #nuevo_nombre = f"{nuevo_nombre_base}_{i}{extension}"
    nuevo_nombre = f"{nuevo_nombre_base}_{i:03}{extension}"  # <- numeración con 3 dígitos
    os.rename(
        os.path.join(directorio, nombre_actual),
        os.path.join(directorio, nuevo_nombre)
    )

print("Renombrado completo.")


Renombrado completo.


In [ ]:
# Recortar silencio al inicio y al final de los archivos

import os
import librosa
import soundfile as sf
import numpy as np

def recortar_silencio(audio_path, top_db=25):
    y, sr = librosa.load(audio_path, sr=None)
    non_silent_intervals = librosa.effects.split(y, top_db=top_db)
    y_trimmed = np.concatenate([y[start:end] for start, end in non_silent_intervals])
    return y_trimmed, sr

def procesar_dataset(input_dir, output_dir, top_db=25):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.wav'):
                input_file = os.path.join(root, file)

                # Estructura relativa para mantener subcarpetas como "Alegria", etc.
                relative_path = os.path.relpath(input_file, input_dir)
                output_file = os.path.join(output_dir, relative_path)

                # Crear subcarpeta si no existe
                output_folder = os.path.dirname(output_file)
                if not os.path.exists(output_folder):
                    os.makedirs(output_folder)

                try:
                    audio_recortado, sr = recortar_silencio(input_file, top_db=top_db)
                    sf.write(output_file, audio_recortado, sr)
                    print(f"✅ Procesado: {relative_path}")
                except Exception as e:
                    print(f"❌ Error en {relative_path}: {e}")

# USO
input_dataset = r'C:\Users\curso\Downloads\Grabaciones_TFM\Dataset_TFM'                   # Tu carpeta original con audios
output_dataset = r'C:\Users\curso\Downloads\Grabaciones_TFM\Dataset_TFM_recortado'        # Carpeta destino con audios recortados

procesar_dataset(input_dataset, output_dataset, top_db=25)


C:\Users\curso\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


✅ Procesado: Alegria\Alegria_001.wav
✅ Procesado: Alegria\Alegria_002.wav
✅ Procesado: Alegria\Alegria_003.wav
✅ Procesado: Alegria\Alegria_004.wav
✅ Procesado: Alegria\Alegria_005.wav
✅ Procesado: Alegria\Alegria_006.wav
✅ Procesado: Alegria\Alegria_007.wav
✅ Procesado: Alegria\Alegria_008.wav
✅ Procesado: Alegria\Alegria_009.wav
✅ Procesado: Alegria\Alegria_010.wav
✅ Procesado: Alegria\Alegria_011.wav
✅ Procesado: Alegria\Alegria_012.wav
✅ Procesado: Alegria\Alegria_013.wav
✅ Procesado: Alegria\Alegria_014.wav
✅ Procesado: Alegria\Alegria_015.wav
✅ Procesado: Alegria\Alegria_016.wav
✅ Procesado: Alegria\Alegria_017.wav
✅ Procesado: Alegria\Alegria_018.wav
✅ Procesado: Alegria\Alegria_019.wav
✅ Procesado: Alegria\Alegria_020.wav
✅ Procesado: Alegria\Alegria_021.wav
✅ Procesado: Alegria\Alegria_022.wav
✅ Procesado: Alegria\Alegria_023.wav
✅ Procesado: Alegria\Alegria_024.wav
✅ Procesado: Alegria\Alegria_025.wav
✅ Procesado: Alegria\Alegria_026.wav
✅ Procesado: Alegria\Alegria_027.wav
✅

In [ ]:
Parámetro clave: top_db
top_db=20: detecta incluso pausas suaves.

top_db=30: más agresivo, corta más.

Ajusta este valor si recorta demasiado o muy poco.

In [ ]:
Buena pregunta: no, el audio no pierde calidad en cuanto a la frecuencia de muestreo o resolución de bits, siempre que utilices correctamente librosa y soundfile como en el ejemplo que te di.

✅ ¿Qué se mantiene igual?
Frecuencia de muestreo (sr): se conserva (por defecto, si no indicas nada, librosa usa 22050 Hz, pero el código que te di usa sr=None para respetar la original).

Resolución (bit depth): soundfile guarda los archivos en 16-bit PCM por defecto, igual que la mayoría de los .wav estándar.

Calidad de audio: la parte hablada no se modifica, solo se recortan los tramos de silencio.

⚠️ ¿Cuándo podría haber "pérdida"?
Si haces downsampling (reduces el sr, como de 44100 Hz a 16000 Hz), sí puedes perder fidelidad en frecuencias altas.

Si cortas mal (top_db mal ajustado), puedes perder pequeñas partes habladas al inicio o final si el umbral es muy agresivo.

Si lo guardas en formato comprimido (como MP3), ahí sí habría pérdida. Pero en .wav, no.

🔍 Recomendación
Para conservar al 100% la calidad:

Usa librosa.load(..., sr=None) como ya lo estás haciendo ✅.

Usa sf.write(..., subtype='PCM_16') si quieres asegurarte del formato:

sf.write(output_file, audio_recortado, sr, subtype='PCM_16')



## ✅ Código completo con estadísticas de recorte

In [ ]:
import os
import librosa
import soundfile as sf
import numpy as np

duraciones_originales = []
duraciones_recortadas = []

def recortar_silencio(audio_path, top_db=25):
    y, sr = librosa.load(audio_path, sr=None)
    duracion_original = librosa.get_duration(y=y, sr=sr)

    non_silent_intervals = librosa.effects.split(y, top_db=top_db)
    y_trimmed = np.concatenate([y[start:end] for start, end in non_silent_intervals])
    duracion_recortada = librosa.get_duration(y=y_trimmed, sr=sr)

    return y_trimmed, sr, duracion_original, duracion_recortada

def procesar_dataset(input_dir, output_dir, top_db=25):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.wav'):
                input_file = os.path.join(root, file)
                relative_path = os.path.relpath(input_file, input_dir)
                output_file = os.path.join(output_dir, relative_path)
                output_folder = os.path.dirname(output_file)

                if not os.path.exists(output_folder):
                    os.makedirs(output_folder)

                try:
                    y_trimmed, sr, dur_orig, dur_trim = recortar_silencio(input_file, top_db)
                    sf.write(output_file, y_trimmed, sr, subtype='PCM_16')

                    duraciones_originales.append(dur_orig)
                    duraciones_recortadas.append(dur_trim)

                    print(f"✅ {relative_path} | Original: {dur_orig:.2f}s -> Recortado: {dur_trim:.2f}s")

                except Exception as e:
                    print(f"❌ Error en {relative_path}: {e}")

def mostrar_resumen():
    if not duraciones_originales:
        print("⚠️ No se procesaron audios.")
        return

    prom_orig = np.mean(duraciones_originales)
    prom_trim = np.mean(duraciones_recortadas)
    reduccion_abs = prom_orig - prom_trim
    reduccion_pct = 100 * reduccion_abs / prom_orig

    print("\n📊 Resumen del recorte de silencios:")
    print(f"Duración promedio original:   {prom_orig:.2f} segundos")
    print(f"Duración promedio recortada:  {prom_trim:.2f} segundos")
    print(f"Reducción promedio absoluta:  {reduccion_abs:.2f} segundos")
    print(f"Reducción promedio relativa:  {reduccion_pct:.2f}%")

# === USO ===
input_dataset = r'C:\Users\curso\Downloads\Grabaciones_TFM\Dataset_TFM'                  # carpeta original
output_dataset = r'C:\Users\curso\Downloads\Grabaciones_TFM\Dataset_TFM_recortado2'      # carpeta destino
procesar_dataset(input_dataset, output_dataset, top_db=25)
mostrar_resumen()


✅ Alegria\Alegria_001.wav | Original: 4.27s -> Recortado: 3.53s
✅ Alegria\Alegria_002.wav | Original: 4.48s -> Recortado: 2.31s
✅ Alegria\Alegria_003.wav | Original: 4.95s -> Recortado: 3.25s
✅ Alegria\Alegria_004.wav | Original: 5.28s -> Recortado: 3.22s
✅ Alegria\Alegria_005.wav | Original: 5.22s -> Recortado: 3.21s
✅ Alegria\Alegria_006.wav | Original: 4.43s -> Recortado: 2.61s
✅ Alegria\Alegria_007.wav | Original: 3.92s -> Recortado: 2.55s
✅ Alegria\Alegria_008.wav | Original: 4.06s -> Recortado: 3.08s
✅ Alegria\Alegria_009.wav | Original: 4.04s -> Recortado: 2.89s
✅ Alegria\Alegria_010.wav | Original: 3.18s -> Recortado: 2.54s
✅ Alegria\Alegria_011.wav | Original: 4.78s -> Recortado: 2.02s
✅ Alegria\Alegria_012.wav | Original: 4.13s -> Recortado: 2.51s
✅ Alegria\Alegria_013.wav | Original: 5.60s -> Recortado: 5.07s
✅ Alegria\Alegria_014.wav | Original: 4.41s -> Recortado: 2.12s
✅ Alegria\Alegria_015.wav | Original: 4.88s -> Recortado: 2.94s
✅ Alegria\Alegria_016.wav | Original: 3.

Los 4 modelos que presentaré son:

Modelo CNN simple

Modelo LSTM

Modelo CNN + LSTM

Modelo basado en Transfer Learning con YAMNet (preentrenado)

### Preprocesamiento común para todos los modelos

In [ ]:
import os
import librosa
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Configuración
emotion_labels = ['Alegria', 'Enojo', 'Miedo', 'Neutral', 'Sorpresa', 'Tristeza']
dataset_path = 'dataset/'
sample_rate = 22050
max_duration = 3  # segundos
n_mfcc = 40

def extract_features(file_path):
    audio, sr = librosa.load(file_path, sr=sample_rate, duration=max_duration, res_type='kaiser_fast')
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfcc.T, axis=0)

X, y = [], []
for label in emotion_labels:
    folder_path = os.path.join(dataset_path, label)
    for file in os.listdir(folder_path):
        if file.endswith('.wav'):
            features = extract_features(os.path.join(folder_path, file))
            X.append(features)
            y.append(label)

X = np.array(X)
y = LabelEncoder().fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Modelo 1: CNN simple

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization

X_train_cnn = X_train[..., np.newaxis]
X_test_cnn = X_test[..., np.newaxis]

model_cnn = Sequential([
    Conv1D(64, 5, activation='relu', input_shape=(n_mfcc, 1)),
    MaxPooling1D(2),
    BatchNormalization(),
    Conv1D(128, 5, activation='relu'),
    MaxPooling1D(2),
    Flatten(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(len(emotion_labels), activation='softmax')
])

model_cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_cnn.fit(X_train_cnn, y_train, epochs=30, batch_size=32, validation_data=(X_test_cnn, y_test))


In [ ]:
✔️ Ventajas:

Rápido de entrenar.

Capta patrones espaciales en MFCC.

❌ Desventajas:

No capta relaciones temporales (dinámicas).

### Modelo 2: LSTM

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Reshape

X_train_lstm = X_train.reshape(-1, 10, 4)
X_test_lstm = X_test.reshape(-1, 10, 4)

model_lstm = Sequential([
    LSTM(64, return_sequences=False, input_shape=(10, 4)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(len(emotion_labels), activation='softmax')
])

model_lstm.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_lstm.fit(X_train_lstm, y_train, epochs=30, batch_size=32, validation_data=(X_test_lstm, y_test))


In [ ]:
✔️ Ventajas:

Captura relaciones temporales de forma efectiva.

Mejor para secuencias largas.

❌ Desventajas:

Más lento de entrenar.

Requiere reshape especial.

### Modelo 3: CNN + LSTM

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout

X_train_cnn_lstm = X_train.reshape(-1, 10, 4)
X_test_cnn_lstm = X_test.reshape(-1, 10, 4)

model_cnn_lstm = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(10, 4)),
    MaxPooling1D(2),
    LSTM(64),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(len(emotion_labels), activation='softmax')
])

model_cnn_lstm.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_cnn_lstm.fit(X_train_cnn_lstm, y_train, epochs=30, batch_size=32, validation_data=(X_test_cnn_lstm, y_test))


In [ ]:
✔️ Ventajas:

Combina lo mejor de CNN y LSTM.

Buen rendimiento en muchas tareas de audio.

❌ Desventajas:

Más complejo y más propenso a overfitting si el dataset es pequeño.

### Modelo 4: Transfer Learning con YAMNet (TensorFlow Hub)

In [ ]:
import tensorflow_hub as hub

yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)

def extract_yamnet_embeddings(file_path):
    waveform, sr = librosa.load(file_path, sr=16000)
    scores, embeddings, _ = yamnet_model(waveform)
    return np.mean(embeddings.numpy(), axis=0)

X_yam, y_yam = [], []
for label in emotion_labels:
    folder_path = os.path.join(dataset_path, label)
    for file in os.listdir(folder_path):
        if file.endswith('.wav'):
            emb = extract_yamnet_embeddings(os.path.join(folder_path, file))
            X_yam.append(emb)
            y_yam.append(label)

X_yam = np.array(X_yam)
y_yam = LabelEncoder().fit_transform(y_yam)
X_train_yam, X_test_yam, y_train_yam, y_test_yam = train_test_split(X_yam, y_yam, test_size=0.2)

# Clasificador final
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(X_train_yam, y_train_yam)
print("Precisión:", rf_model.score(X_test_yam, y_test_yam))


In [ ]:
✔️ Ventajas:

Potente representación del audio (preentrenado).

Bueno para generalizar y transferir a nuevos contextos.

❌ Desventajas:

Requiere conexión a internet o descarga previa.

Puede no estar afinado para emociones específicamente.

In [ ]:
✅ Recomendación general
Para datasets pequeños: Modelo 4 (Transfer Learning) o CNN simple.

Para relaciones temporales complejas: Modelo 2 o Modelo 3.

Para producción rápida: CNN.

Para máxima precisión con recursos: CNN + LSTM o Transfer Learning + clasificador robusto.

In [ ]:
### codigo ideal con YAMNet

In [ ]:

# ✅ Requiere instalación previa:
# pip install librosa tensorflow tensorflow_hub scikit-learn soundfile

import os
import numpy as np
import librosa
import tensorflow_hub as hub
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import joblib

# --- Configuración ---
DATASET_PATH = 'dataset'  # Estructura: dataset/Alegria/, dataset/Enojo/, etc.
EMOTIONS = ['Alegria', 'Enojo', 'Miedo', 'Neutral', 'Sorpresa', 'Tristeza']
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'

# --- Cargar modelo YAMNet ---
print("Cargando modelo YAMNet...")
yamnet_model = hub.load(yamnet_model_handle)

# --- Función para extraer embeddings ---
def extract_yamnet_embedding(file_path):
    try:
        waveform, sr = librosa.load(file_path, sr=16000)
        scores, embeddings, spectrogram = yamnet_model(waveform)
        return np.mean(embeddings.numpy(), axis=0)
    except Exception as e:
        print(f"Error en {file_path}: {e}")
        return None

# --- Extraer características ---
X = []
y = []

print("Extrayendo embeddings de YAMNet...")
for emotion in EMOTIONS:
    emotion_path = os.path.join(DATASET_PATH, emotion)
    for fname in os.listdir(emotion_path):
        if fname.endswith(".wav"):
            file_path = os.path.join(emotion_path, fname)
            embedding = extract_yamnet_embedding(file_path)
            if embedding is not None:
                X.append(embedding)
                y.append(emotion)

X = np.array(X)
y = LabelEncoder().fit_transform(y)

# --- División entrenamiento/prueba ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# --- Entrenamiento con Random Forest ---
print("Entrenando clasificador Random Forest...")
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# --- Evaluación ---
y_pred = clf.predict(X_test)
print("\n--- Reporte de clasificación ---")
print(classification_report(y_test, y_pred, target_names=EMOTIONS))

print("\n--- Matriz de confusión ---")
print(confusion_matrix(y_test, y_pred))

# --- Validación cruzada (opcional) ---
# scores = cross_val_score(clf, X, y, cv=5)
# print(f"Precisión media (cross-validation): {np.mean(scores):.2f}")

# --- Guardar modelo entrenado ---
joblib.dump(clf, "modelo_emociones_yamnet.pkl")
print("✅ Modelo guardado como modelo_emociones_yamnet.pkl")


In [ ]:
Aquí tienes el archivo Python completo para ejecutar el modelo de reconocimiento de emociones usando YAMNet + Random Forest:

📥 Descargar yamnet_rf_emotion_recognition.py

Este script:

Carga tus audios por carpeta/emoción.

Extrae los embeddings usando YAMNet (preentrenado por Google).

Entrena un clasificador Random Forest.

Muestra métricas de desempeño y guarda el modelo entrenado.

In [ ]:
### ¿Deseas que el modelo guardado también pueda ser cargado después para predicciones nuevas?

In [ ]:
✅ Código para cargar el modelo y predecir la emoción
Agrega este bloque en un nuevo script (o al final del existente si lo deseas):

In [ ]:
# === Cargar modelo y predecir una emoción ===
import joblib
import librosa
import tensorflow_hub as hub
import numpy as np

# Configuración
modelo_path = "modelo_emociones_yamnet.pkl"
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)
EMOTIONS = ['Alegria', 'Enojo', 'Miedo', 'Neutral', 'Sorpresa', 'Tristeza']

# Función para extraer embedding del nuevo audio
def extract_embedding(file_path):
    waveform, sr = librosa.load(file_path, sr=16000)
    scores, embeddings, spectrogram = yamnet_model(waveform)
    return np.mean(embeddings.numpy(), axis=0)

# Ruta del nuevo archivo a predecir
nuevo_audio = "ruta/del/audio.wav"  # <-- CAMBIAR ESTO

# Cargar modelo entrenado
modelo = joblib.load(modelo_path)

# Predecir emoción
embedding = extract_embedding(nuevo_audio).reshape(1, -1)
prediccion = modelo.predict(embedding)[0]
print(f"🔍 Emoción predicha: {EMOTIONS[prediccion]}")


In [ ]:
🧠 ¿Qué hace este código?
Carga el modelo .pkl guardado.

Usa YAMNet para extraer el embedding del nuevo audio.

Aplica el clasificador y devuelve la emoción predicha.

🛠️ Requisitos
Este bloque funciona con los mismos paquetes:

pip install librosa tensorflow tensorflow_hub scikit-learn soundfile joblib

In [ ]:
### ¿se puede usar otro de los modelos indicados anteriormente para hacer el reconocimiento de emociones?

In [ ]:
Sí, puedes usar cualquiera de los modelos que vimos anteriormente (CNN, LSTM, CNN+LSTM o Transfer Learning con YAMNet) para el reconocimiento de emociones, pero algunas opciones son más adecuadas que otras dependiendo del tamaño de tu dataset, el contexto de uso y la precisión que busques.

Vamos a repasar cómo puedes usar cada uno de ellos para predicción en producción, y cuál es más recomendable en tu caso.

✅ 1. Transfer Learning con YAMNet (RECOMENDADO para tu caso)
✔️ Funciona bien con datasets pequeños.

✔️ Muy buena capacidad de generalización.

✔️ Rápido de entrenar.

✔️ Embedding único → fácil de predecir.

Ya te generé código para:

Entrenamiento (yamnet_rf_emotion_recognition.py)

Predicción (predict_yamnet_emotion.py propuesto)

✅ 2. CNN simple sobre MFCC
✔️ Ventajas:
Fácil de implementar.

Razonablemente preciso si los datos son limpios.

❗ Para predicción:
Cargas el modelo .h5 con keras.models.load_model.

Extraes MFCC del nuevo .wav igual que en el entrenamiento.

Redimensionas el input para que tenga forma (1, 40, 1).

📌 Código ejemplo de predicción:

In [ ]:
from tensorflow.keras.models import load_model
import librosa
import numpy as np

model = load_model("modelo_cnn_emociones.h5")
EMOTIONS = ['Alegria', 'Enojo', 'Miedo', 'Neutral', 'Sorpresa', 'Tristeza']

def predict_emotion(file_path):
    y, sr = librosa.load(file_path, sr=22050, duration=3, res_type='kaiser_fast')
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    mfcc_mean = np.mean(mfcc.T, axis=0).reshape(-1, 40, 1)
    pred = model.predict(mfcc_mean)
    return EMOTIONS[np.argmax(pred)]

print(predict_emotion("ruta/a/audio.wav"))


In [ ]:
✅ 3. LSTM o CNN + LSTM sobre MFCC
✔️ Ventajas:
Capta mejor la temporalidad.

Útil si las emociones están ligadas a cambios temporales.

❗ Consideraciones:
Necesitas reshaping como (1, 10, 4) si trabajaste así en el entrenamiento.

Mayor riesgo de sobreajuste si el dataset es pequeño.

❌ ¿Cuándo no usar CNN/LSTM directamente?
Cuando tienes menos de 500–1000 audios por clase.

Si no tienes acceso a buena validación cruzada.

Si deseas hacer inferencia rápida o desde dispositivos ligeros.

🏁 Recomendación final para predicción
Modelo	¿Recomendado para tu dataset?	¿Uso en producción fácil?	Notas
YAMNet + RF	✅ Sí	                     ✅ Muy fácil	Ideal con 250 muestras/clase
CNN (MFCC)	⚠️ Aceptable	              ✅ Sí	Mejor con >500 por clase
LSTM (MFCC)	⚠️ Aceptable	              ⚠️ Moderado	Riesgo de sobreajuste
CNN + LSTM	⚠️ Potente pero arriesgado	⚠️ Más complejo	Solo si controlas el overfitting


